In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append("lib")

import lib

import torch
torch.set_default_dtype(torch.float64)

from functorch import vmap

In [2]:
coords_x, coords_y, _ = lib.utils.read_coords("2d_burger_data/time_step_0.csv")
coords = torch.stack((coords_x, coords_y), dim=1)

In [3]:
num_steps = 501
num_nodes = coords_x.shape[0]

datas = lib.utils.read_data(num_steps, num_nodes, '2d_burger_data/time_step_', 'vel_0')
datas.shape

torch.Size([501, 14641])

In [4]:
N = datas.shape[1]
n = 20
mu = int(np.ceil(N/40))
neighbours = lib.utils.topk_neighbours(coords, mu)

In [5]:
nrbs = lib.nrbs.NRBS(N=N, n=n, mu=mu, neighbours=neighbours)

In [6]:
encoded = nrbs.encode(datas)
decoded = nrbs.decode(encoded)

bubbles shape:  torch.Size([20, 14641, 367])


In [7]:
vmap_bubble = vmap(nrbs.bubble, in_dims=0)
vmap_vmap_bubble = vmap(vmap_bubble, in_dims=0)
bubbles = vmap_vmap_bubble(nrbs.bandwidth)

In [8]:
node_idxs = torch.linspace(0, 10, 10, dtype=torch.long)

In [9]:
nrbs.smooth_vec(0, node_idxs, bubbles)

tensor([[-0.0049],
        [-0.0015],
        [ 0.0063],
        [-0.0027],
        [-0.0051],
        [ 0.0059],
        [-0.0074],
        [-0.0049],
        [-0.0094],
        [-0.0055]], grad_fn=<SumBackward1>)

In [10]:
nrbs.smooth(0, 0, bubbles=bubbles)

tensor(-0.0049, grad_fn=<SumBackward0>)

In [11]:
nrbs.smooth_basis(bubbles=bubbles).shape

torch.Size([20, 14641])

In [12]:
encoded.shape

torch.Size([501, 20])

In [15]:
decoded.shape

torch.Size([501, 14641])